# Volume 1, Chapter 7: Context Management

**Handle Large Configs That Exceed Token Limits**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/main/Volume-1-Foundations/Colab-Notebooks/Vol1_Ch7_Context_Management.ipynb)

---

**What you'll learn:**
- 📏 Check if your config fits in context
- ✂️ Chunk large configs intelligently
- 🔄 Map-Reduce pattern for analysis
- 💾 Cache prompts to save money

**Time:** ~15 minutes | **Cost:** ~$0.05

## 🔧 Setup

In [ ]:
!pip install -q anthropic tiktoken

import os
from getpass import getpass

try:
    from google.colab import userdata
    os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
except:
    if 'ANTHROPIC_API_KEY' not in os.environ:
        os.environ['ANTHROPIC_API_KEY'] = getpass('Anthropic API key: ')

from anthropic import Anthropic
import tiktoken

client = Anthropic()
encoding = tiktoken.get_encoding("cl100k_base")

def count_tokens(text):
    return len(encoding.encode(text))

print("✓ Ready!")

---
## 📏 Example 1: Check If Config Fits

In [ ]:
# Context window limits
LIMITS = {
    "claude-3-5-sonnet": 200_000,
    "claude-3-5-haiku": 200_000,
    "gpt-4o": 128_000,
}

def will_fit(text, model="claude-3-5-sonnet", output_reserve=4000, prompt_overhead=500):
    """Check if content fits in context window."""
    tokens = count_tokens(text)
    limit = LIMITS[model]
    available = limit - output_reserve - prompt_overhead
    
    return {
        "fits": tokens <= available,
        "content_tokens": tokens,
        "available_tokens": available,
        "utilization": (tokens / available) * 100,
        "overflow": max(0, tokens - available)
    }

# Create sample configs of different sizes
small_config = """hostname R1
interface Gi0/0
 ip address 10.1.1.1 255.255.255.0
"""

# Medium config (~500 lines)
medium_config = (small_config + "!\n") * 100

# Large config (~5000 lines) 
large_config = medium_config * 10

# Huge config (~50000 lines)
huge_config = large_config * 10

print("📏 CONTEXT WINDOW CHECK")
print("=" * 60)

configs = [
    ("Small (~50 lines)", small_config),
    ("Medium (~500 lines)", medium_config),
    ("Large (~5K lines)", large_config),
    ("Huge (~50K lines)", huge_config),
]

for name, config in configs:
    result = will_fit(config)
    status = "✅ FITS" if result["fits"] else f"❌ OVERFLOW ({result['overflow']:,} tokens)"
    print(f"\n{name}:")
    print(f"  Tokens: {result['content_tokens']:,}")
    print(f"  Utilization: {result['utilization']:.1f}%")
    print(f"  Status: {status}")

---
## ✂️ Example 2: Intelligent Chunking by Interface

In [ ]:
import re

def chunk_by_interface(config):
    """Split config into interface blocks."""
    chunks = []
    
    # Split on 'interface' keyword
    parts = re.split(r'(^interface \S+)', config, flags=re.MULTILINE)
    
    # Combine interface name with its config
    current_chunk = ""
    for part in parts:
        if part.startswith('interface '):
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = part
        else:
            current_chunk += part
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

# Test config
test_config = """hostname ROUTER-01
!
interface GigabitEthernet0/0
 description WAN
 ip address 203.0.113.1 255.255.255.252
 no shutdown
!
interface GigabitEthernet0/1
 description LAN
 ip address 192.168.1.1 255.255.255.0
 no shutdown
!
interface Loopback0
 ip address 1.1.1.1 255.255.255.255
!
router ospf 1
 network 192.168.1.0 0.0.0.255 area 0
"""

chunks = chunk_by_interface(test_config)

print("✂️ CHUNKED BY INTERFACE")
print("=" * 60)
print(f"Total chunks: {len(chunks)}")

for i, chunk in enumerate(chunks):
    print(f"\n--- Chunk {i+1} ({count_tokens(chunk)} tokens) ---")
    print(chunk[:200] + ("..." if len(chunk) > 200 else ""))

---
## 🔄 Example 3: Map-Reduce Pattern

In [ ]:
def analyze_chunk(chunk, chunk_name):
    """Analyze a single config chunk (MAP step)."""
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=300,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Analyze this config section for security issues. Be brief.

Section: {chunk_name}
Config:
{chunk}

Return: List any issues found, or "No issues" if clean."""
        }]
    )
    return response.content[0].text

def combine_results(results):
    """Combine chunk analyses (REDUCE step)."""
    all_results = "\n\n".join([f"**{name}:**\n{result}" for name, result in results])
    
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=500,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Summarize these security findings into a final report:

{all_results}

Provide:
1. Critical issues (if any)
2. Warnings
3. Overall risk level (Low/Medium/High)"""
        }]
    )
    return response.content[0].text

# Full config to analyze
full_config = """hostname BRANCH-01
!
enable password cisco123
!
interface GigabitEthernet0/0
 description WAN
 ip address 203.0.113.1 255.255.255.252
!
interface GigabitEthernet0/1
 description LAN
 ip address 192.168.1.1 255.255.255.0
!
line vty 0 4
 transport input telnet
 password cisco
!
snmp-server community public RO
"""

print("🔄 MAP-REDUCE ANALYSIS")
print("=" * 60)

# MAP: Analyze each chunk
chunks = chunk_by_interface(full_config)
chunk_results = []

for i, chunk in enumerate(chunks):
    name = f"Section {i+1}"
    if "interface" in chunk:
        match = re.search(r'interface (\S+)', chunk)
        if match:
            name = match.group(1)
    elif "line vty" in chunk:
        name = "VTY Lines"
    elif "snmp" in chunk:
        name = "SNMP Config"
    
    print(f"\n📍 Analyzing: {name}...")
    result = analyze_chunk(chunk, name)
    chunk_results.append((name, result))
    print(f"   Result: {result[:100]}...")

# REDUCE: Combine results
print("\n" + "=" * 60)
print("📊 COMBINED REPORT")
print("=" * 60)
final_report = combine_results(chunk_results)
print(final_report)

---
## 📦 Example 4: Chunk by Token Limit with Overlap

In [ ]:
def chunk_by_tokens(text, max_tokens=1000, overlap_tokens=100):
    """Chunk text by token limit with overlap for context continuity."""
    lines = text.split('\n')
    chunks = []
    current_chunk = []
    current_tokens = 0
    
    for line in lines:
        line_tokens = count_tokens(line)
        
        if current_tokens + line_tokens > max_tokens and current_chunk:
            # Save current chunk
            chunks.append('\n'.join(current_chunk))
            
            # Start new chunk with overlap (last few lines)
            overlap_lines = []
            overlap_count = 0
            for prev_line in reversed(current_chunk):
                if overlap_count + count_tokens(prev_line) <= overlap_tokens:
                    overlap_lines.insert(0, prev_line)
                    overlap_count += count_tokens(prev_line)
                else:
                    break
            
            current_chunk = overlap_lines + [line]
            current_tokens = overlap_count + line_tokens
        else:
            current_chunk.append(line)
            current_tokens += line_tokens
    
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    
    return chunks

# Test with medium config
print("📦 TOKEN-BASED CHUNKING WITH OVERLAP")
print("=" * 60)

chunks = chunk_by_tokens(medium_config, max_tokens=500, overlap_tokens=50)

print(f"Original: {count_tokens(medium_config):,} tokens")
print(f"Chunks: {len(chunks)}")
print()

for i, chunk in enumerate(chunks[:3]):  # Show first 3
    print(f"Chunk {i+1}: {count_tokens(chunk)} tokens")
print(f"... and {len(chunks)-3} more chunks")

---
## 🎯 Example 5: Smart Chunking Decision

In [ ]:
def smart_analyze(config):
    """Automatically decide chunking strategy based on size."""
    tokens = count_tokens(config)
    
    print(f"Config size: {tokens:,} tokens")
    
    if tokens < 50000:  # Fits in context
        print("Strategy: DIRECT (fits in context)")
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1000,
            temperature=0,
            messages=[{
                "role": "user",
                "content": f"Analyze for security issues:\n{config}"
            }]
        )
        return response.content[0].text
    
    elif tokens < 200000:  # Needs chunking
        print("Strategy: CHUNK + MAP-REDUCE")
        chunks = chunk_by_interface(config)
        if len(chunks) < 3:
            chunks = chunk_by_tokens(config, max_tokens=40000)
        
        results = []
        for i, chunk in enumerate(chunks):
            response = client.messages.create(
                model="claude-3-5-haiku-20241022",
                max_tokens=300,
                temperature=0,
                messages=[{
                    "role": "user",
                    "content": f"List security issues in this config section (brief):\n{chunk}"
                }]
            )
            results.append(response.content[0].text)
        
        return "\n\n".join(results)
    
    else:  # Too large
        print("Strategy: SUMMARIZE FIRST")
        return "Config too large. Consider splitting by device or section first."

# Test
print("🎯 SMART ANALYSIS")
print("=" * 60)
result = smart_analyze(test_config)
print("\nResult:")
print(result)

---
## 🎯 Key Takeaways

| Config Size | Strategy | Notes |
|-------------|----------|-------|
| < 50K tokens | Direct analysis | Single API call |
| 50K - 200K | Chunk + Map-Reduce | Split, analyze, combine |
| > 200K | Summarize first | Or use Gemini (2M context) |

**Chunking best practices:**
1. Chunk at natural boundaries (interfaces, sections)
2. Use overlap to maintain context
3. Use cheaper model (Haiku) for chunk analysis
4. Combine results with final summary

---

## 📚 Next Steps

➡️ [Chapter 8: Cost Optimization](./Vol1_Ch8_Cost_Optimization.ipynb)